# Project: **Rootfinding using the Newton-Raphson method** in the **GNU Scientific Library (GSL)**

The Newton-Raphson rootfinding method is implemented in the GNU Scientific Library (GSL) as one of several rootfinding methods available to the developer.

### About the method
The Newton-Raphson method is a classical and relatively simple method of calculating the root(s) of a differentiable function using the first-order derivatives of a function. The theory behind this method is that the intersection of a  line tangent to a point on the function and the function itself is generally closer to the root of the function than the original point. Mathematically, the Newton-Raphson method is iterative and is defined as 
$$x_n = x_{n-1}$$  

with $x_0$ being pre-defined by, in this case, the developer. Graphically, Newton-Raphson iterations look like the below image  
<img src="images/newton_iterations.gif" alt="a gif of newton iterations in action" style="width: 400px;"/>  
_GIF courtesy [Ralf Pfeifer](https://commons.wikimedia.org/w/index.php?curid=2268473)_

The convergence of the Newton-Raphson method is locally quadratic. However, there are a number of considerations which may make the use of this method slow or infeasible. 
- If the function the developer is using is non-differentiable, then it will be impossible to find any derivative to use in the method. Finding the derivative of a function is not terribly easy either, so it may be wise on more complex polynomials to identify a different method of solving for the root.
- Even if the funciton is differentiable, the method may overshoot/diverge, as it does, for example, for the cube root [[1]](http://www.cas.mcmaster.ca/~cs4te3/notes/newtons_method.pdf). 
- The method may also oscillate by bouncing between two points [[2]](http://sites.millersville.edu/bikenaga/calculus/newton/newton.html), never converging and instead wasting computing time and resources if the programmer does not put in failsafes to cancel the operation.   
Overall, the speed and efficiency of the method is largely dependent on making a good initial guess [[3]](https://github.com/cu-numcomp/numcomp-class/blob/master/Rootfinding.ipynb). The GSL accounts for these issues by including a number of other rootfinding methods that don't encounter these issues but may take a longer time to solve. 

### About the software
![GNU logo](images/gnu_small.png)
The GSL is an extensive numerical computation library with over 1,000 mathematical functions and associated tests. The library provides baseline numerical methods to several other scientific libraries and has wrappers in a number of languages other than C, including Ruby, Python, Julia, and Rust. The latest version was released in August of 2019 under the GNU General Public License. GSL powers simulation and scientific software at organizations such as NASA, Microsoft, and Apple and is a key component in the architecture of these packages. 

### Method as it appears in the software
The GSL software has definitions for several different types of rootfinding solvers, allowing developers to "plug and play" by choosing which one to use and specififying it when defining the solver. As such, when defining the solver, the developer must provide the type. Due to the Newton-Raphson algorithm's use of derivatives, it is defined as an `gsl_root_fdfsolver_type`, specifically `gsl_root_fdfsolver_newton`.  
Each of the iterative solvers used within the library are defined with an initialize and an iterate method - in our case, `newton_init` and `newton_iterate` respectively. The state of the iteration is maintained and tracked with the following struct
```c
typedef struct
  {
    double f, df;
  }
newton_state_t;
```
where f and df are the current point evaluated for the function and derivative respectively.   
The initial point for the method to start on is defined by the developer in the initialization function.
```c
static int newton_init (void * vstate, gsl_function_fdf * fdf, double * root)
```  
On each iteration, the function and its derivative must be passed as a `gsl_function_fdf` and the current state and point are passed as the above `newton_state_t` and a double pointer. 
```c
static int newton_iterate (void * vstate, gsl_function_fdf * fdf, double * root)
```  
Within the `newton_iterate` function, we find the core of the method at   
```c
root_new = *root - (state->f / state->df);

*root = root_new ;
```  
This iteration will continue until a root is reached or an error is encountered.

### Further questions
- What was the design justification of not storing $f(x)$ and $df(x)$ and instead requiring the developer/library to pass it for each iteration?
- What methods besides bisection would allow us to make a good initial guess for Newton-Raphson?

### References
- [GSL Documentation](https://www.gnu.org/software/gsl/doc/latex/gsl-ref.pdf) - The documentation for `gsl_root_fdfsolver_newton` can be found on page 407. The snippet referenced throughout this project: 
![doc snippet](images/doc_string.png)
- [newton.c](https://github.com/ampl/gsl/blob/master/roots/newton.c) - GSL Source for the Newton rootfinding method.